In [1]:
# Author : Ravi Ranjan
# IIT Bombay 2018-2020

import json
import numpy as np

In [2]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [3]:
with open('val_sceneGraphs.json') as f:
    gqa_vg_train = json.load(f)

In [4]:
temp_dict = {}
# number of images in the catalog = N
N = 100
i = 0
for idx, record in gqa_vg_train.items():
    temp_dict[idx] = record
    i += 1
    if i == N:
        break
gqa_vg_train = temp_dict
len(gqa_vg_train)

100

In [5]:
# loading all the vocabularies 
obj2ind=np.load('object_vocab.npy',allow_pickle=True).item()
attr2ind=np.load('attr_vocab.npy',allow_pickle=True).item()
rel2ind=np.load('relation_vocab.npy',allow_pickle=True).item()
redundant=np.load('redundant_obj_dict.npy', allow_pickle=True).item()

In [6]:
# checking max. number of objects in a scene graph : 
maximum = -9999
for idx, record in gqa_vg_train.items():
    maximum = max(maximum, len(record['objects']))
maximum

37

In [7]:
max_num_objects_in_sg = 40
pad_attribute=attr2ind['<pad>']
pad_attribute

167

In [8]:
# checking max. number of objects in a scene graph : 
maximum = -9999
for idx, record in gqa_vg_train.items():
    attr_set = set()
    for obj_id, obj_record in record['objects'].items():
        for attribute in obj_record['attributes']:
            attr_set.add(attribute)
    maximum = max(maximum, len(attr_set))
maximum

17

In [9]:
max_num_attrs_in_sg = 20


In [10]:
# structure needed : 
#     for each sg
#         i need one array of objects. this array needs to have fixed size
#         need to pad the remaining places if number of objects < fixed size
#         also need object_mask for telling till which place, there are valid objects
        
#         need to do the same thing for attributes
        
#         also need to create adj matrix for representing tuples of obj-rel-obj and obj-rel-attr
        

In [11]:
n_items = take(1, redundant.items())
print(n_items)

[('2716708', 'nose')]


In [24]:
object_masks = []
attr_masks = []
image_objects = []
image_attrs = []
image_obj_obj_edges = []
image_obj_attr_edges = []
dummy_rel = 'dummy'

In [32]:
for idx, sceneGraph in gqa_vg_train.items():
    # creation of image_objects list
    objects = np.ones(max_num_objects_in_sg)*pad_attribute
    index = 0
    for obj_id, obj_record in sceneGraph['objects'].items():
        objects[index] = obj2ind[redundant[obj_id]]
        index += 1
    # number of objects per datapoint(query scene graph)
#     print(objects)
    obj_mask=np.zeros(max_num_objects_in_sg)
    obj_mask[:(index)]=1
#     print(obj_mask)
    object_masks.append(obj_mask)    
    # creation of object masks done
    image_objects.append(objects)
#     ================================================================================
    
    # creation of image_attr_list
    attributes = np.ones(max_num_attrs_in_sg)*pad_attribute
    for obj_id, obj_record in sceneGraph['objects'].items():
        index = 1
        for attr_name in obj_record['attributes']:
            attributes[index] = attr2ind[attr_name]
            index += 1
    # number of attrs per datapoint(query scene graph)
#     print(attributes)
    attr_mask=np.zeros(max_num_attrs_in_sg)
    attr_mask[:(index)]=1
#     print(attr_mask)
    exit(1)
    attr_masks.append(attr_mask)
    # creation of attr masks done
    image_attrs.append(attributes)
#     ===================================================================================

    # creation of edges 
    
    # creation of obj-rel-obj edges
    overall_num_objects = len(obj2ind)
    adj_mat_obj_obj = np.zeros((overall_num_objects, overall_num_objects))
    for obj_id, obj_record in sceneGraph['objects'].items():
        obj1 = obj2ind[obj_record['name']]
        for relation in obj_record['relations']:
            obj2 = obj2ind[redundant[relation['object']]]
            rel = rel2ind[relation['name']]
            adj_mat_obj_obj[obj1, obj2] = rel
    image_obj_obj_edges.append(adj_mat_obj_obj)
#     ===================================================================================

    
    # creation of obj-rel-attr edges 
    overall_num_attrs = len(attr2ind)
    adj_mat_obj_attr=np.zeros((overall_num_objects,overall_num_attrs))
    for obj_id, ob_record in sceneGraph['objects'].items():
        for attr_name in obj_record['attributes']:
            obj = obj2ind[obj_record['name']]
            rel = rel2ind[dummy_rel]
            attr = attr2ind[attr_name] 
            adj_mat_obj_obj[obj, attr] = rel
    image_obj_attr_edges.append(adj_mat_obj_attr)

In [33]:
adj_mat_obj_obj.shape

(422, 422)

In [34]:
adj_mat_obj_attr.shape

(422, 167)

In [35]:
print(len(image_objects))
print(len(image_attrs))
print(len(image_obj_obj_edges))
print(len(image_obj_attr_edges))


400
400
400
400


In [37]:
query_scene_graph={'object-nodes' : image_objects, 'attribute-nodes' : image_attrs, 
                   'obj-obj-edges' : image_obj_obj_edges, 'obj-attr-edges' : image_obj_attr_edges,
                   'object-node-masks' : object_masks, 'attr-node-masks' : attr_masks}

np.save('query_scene_graph'+'train'+'.npy',query_scene_graph)
